In [1]:
from random import random,sample
import numpy as np
import time
import math
import matplotlib.pyplot as plt
from haversine import haversine

In [2]:
def tsp_read(nodes):
    infile = open(nodes, 'r')
    content = infile.readline().strip().split()
    print("File Name: ", content[2])

    while content[0] != 'NODE_COORD_SECTION':
        if(content[0] == 'DIMENSION'):
            dimension = content[2]
        content = infile.readline().strip().split()
    nodelist = []
    placelist = []
    print('Dimension', dimension)
    N = int(dimension)
    for i in range(0, N):
        x, y, z = infile.readline().strip().split()[:]
        nodelist.append([float(y), float(z)])
        placelist.append(x)
    
    infile.close()
    return nodelist, placelist

In [3]:
def euclidean_distance(nodes, n1, n2):
    distance = math.sqrt((nodes[n1][0]-nodes[n2][0])**2 + (nodes[n1][1]-nodes[n2][1])**2)
    return distance

In [4]:
def global_distance(nodes,n1,n2):
    distance = haversine((nodes[n1][0],nodes[n1][1]), (nodes[n2][0],nodes[n2][1]), unit='mi')
    return distance

In [5]:
def cooling_function(T):
    return T * 0.9999

In [6]:
def get_path_cost(dist, path):
    cost = 0
    length = len(path)
    for i in range(length):
        cost = cost + dist[path[i], path[(i+1) % length]]
    return cost

In [7]:
def part_reversal(path, i, j):
    while (j > i):
        path[i], path[j] = path[j], path[i]
        i += 1
        j -= 1
    return path

In [8]:
def simulated_annealing(nodes, dist, n):
    # Initial Path
    path = np.random.permutation(n)
    path_cost = get_path_cost(dist, path)
    initial_path = path
    print("path cost before simulated annealing ", path_cost)
    iteration = 1000000
    temperature = 10000
    ran = 0.5
    for i in range(iteration):
        rev = sample(range(len(nodes)),2)
        newpath = path.copy()
        newpath[rev[0]:rev[1]+1] = newpath[rev[0]:rev[1]+1][::-1] # reverse sublist
        new_pathcost = get_path_cost(dist, newpath)
        
        delE = path_cost - new_pathcost
        try:
            prob = 1 / (1 + math.exp(-delE / temperature))
        except OverflowError:
            prob = 0
        if delE > 0:
            path = newpath.copy()
            path_cost = new_pathcost
        elif random() < prob:
            path = newpath.copy()
            path_cost = new_pathcost
        temperature = cooling_function(temperature)
        if(temperature < 1e-8):
            print(i)
            break
    return path, path_cost, initial_path

In [9]:
def graph(nodes, path, iterator, j, axs):
    X = []
    Y = []
    length = len(path)
    for i in range(length):
        X.append(nodes[path[i]][0])
        Y.append(nodes[path[i]][1])
    X.append(nodes[path[0]][0])
    Y.append(nodes[path[0]][1])
    axs[iterator, j].plot(X,Y)


In [ ]:
def main():
    datalist = [
        "Data/Rajasthan.tsp", 
        "Data/xqf131.tsp",
        "Data/xqg237.tsp", 
        "Data/pma343.tsp",
        "Data/pka379.tsp",
        "Data/bcl380.tsp",
        "Data/pbl395.tsp",
        "Data/pbk411.tsp",
        "Data/pbn423.tsp",
        "Data/pbm436.tsp",
        "Data/xql662.tsp",
        "Data/rbx711.tsp",
        "Data/rbu737.tsp",
        "Data/dkg813.tsp",
        "Data/lim963.tsp",
        "Data/pbd984.tsp",
        "Data/xit1083.tsp",
        "Data/dka1376.tsp",
        "Data/dca1389.tsp",
        "Data/dja1436.tsp",
        "Data/icw1483.tsp"
    ]
    fig = plt.figure(figsize=(10, 10))
    gs = fig.add_gridspec(len(datalist), 2, hspace = 0.3)
    axs = gs.subplots()
    fig.suptitle("DataSets")
    iterator = 0
    for d in datalist:
        nodes, place = tsp_read(d)
        coords = np.array(nodes) 
        n = len(coords)
        # Distance Array
        dist = np.zeros((n, n), dtype=float)

        for i in range(n):
            for j in range(i+1, n):
                if d == "Data/Rajasthan.tsp":
                    dist[i, j] = global_distance(nodes, i, j)
                else:
                    dist[i, j] = euclidean_distance(nodes, i, j)
                dist[j, i] = dist[i, j]

        start = time.time_ns()
        best_path, best_cost, initial_path = simulated_annealing(nodes, dist, n)
        end = time.time_ns()
        print('Execution Time:', end-start)
        print('Initial Path:', initial_path)
        print('Best Path:', best_path)
        print('Best Cost:', best_cost)
        graph(nodes, initial_path,iterator, 0, axs)
        graph(nodes, best_path, iterator, 1, axs)
        iterator += 1
        plt.show


if __name__ == "__main__":
    main()

File Name:  Rajasthan
Dimension 20
path cost before simulated annealing  3367.4685437963903
276296
Execution Time: 2557286312
Initial Path: [ 9 14 16 17 12 15  6  8 18  5 11 10  0 19  4 13  3  2  1  7]
Best Path: [ 4  1 17 13 16  7 14 18 10  9  8  2  5 12  0 19  3  6 11 15]
Best Cost: 1297.0783650384117
File Name:  xqf131
Dimension 131
path cost before simulated annealing  4792.603770786828
276296
Execution Time: 9981289272
Initial Path: [ 44  97  50   3  48  54  83  16  26  53 103 124  81  96  18  60  76  33
  86  66  85 104  99 114  67  47  35  40 112  49  13 105 121 119  32  14
  72 130  95  10 109  69 116 125  39  36  25  59 102   0  77 129  30   6
  88  75  65 113   7  42 120  62  98 108  73  12  45 123  82  57 126  38
  24  37 127   9  43  58  19   4  92   5  61 110  91  46 111  27  17  80
  34 118  11 107   8  89  23  70  56   2  55 106  29  52  93  28  51  84
  63  71 115 128  90  21 101  74  15 117  22  64  79   1 100  20  78  94
 122  68  41  87  31]
Best Path: [ 92  86  87  